<a href="https://colab.research.google.com/github/FNS-Division/geopython-2025/blob/main/0_get_open_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting open data

Welcome to this hands-on session where we'll learn how to analyze infrastructure data using Python. We'll be working with real data to understand how to obtain, process and prepare different layers of infrastructure data. This tutorial will teach you how to handle geographic data, create visualizations for exploratory data analysis, and standardize infrastructure data for analysis.

## Setting up our environment

We start by importing the Python libraries we'll need for our analysis:
- geopandas and shapely: For handling geographic data and operations
- pandas: For data manipulation and analysis
- matplotlib and contextily: For creating visualizations and adding map backgrounds
- osmnx: For accessing OpenStreetMap data
- Other utility libraries for various tasks like generating UUIDs and handling country codes

In [ ]:
!pip install osmnx contextily summarytools pycountry s3fs

In [ ]:
# Standard library imports
import os
import math
import gzip
import shutil

# Data manipulation and analysis
import pandas as pd
import numpy as np
import uuid
import s3fs

# Geospatial libraries
import geopandas as gpd
import osmnx as ox
from shapely.ops import unary_union
import pycountry

# Visualization libraries
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import contextily as cx
import folium

# Interactive tools and display
import ipywidgets as widgets
from IPython.display import display, HTML
from summarytools import dfSummary

We set `fetch_data = False` to use pre-downloaded data instead of fetching it live during the tutorial.

In [ ]:
fetch_data = True

## Get country boundaries

Before analyzing infrastructure within our selected country, we need to define the country's boundaries. We:
1. Load a GeoJSON file containing global UN-recognized country boundaries using geopandas
2. Filter to get just the country boundary
3. Calculate the country's bounding box and UTM projection zone for later use
4. Get the ISO3 country code for standardization

This boundary data will be crucial for clipping our infrastructure data and ensuring we're analyzing points within the country's borders.

**We suggest working with Sao Tome and Principe, Barbados, Bahrain or Samoa.**

In [ ]:
# @title Select country
# Define layout
item_layout2 = widgets.Layout(
    width='auto',
    min_width='20px',
    flex='1 1',
    display='flex',
    flex_flow='row wrap',
    align_items='center',
    justify_content='space-between'
)

# Country dropdown with all countries as options
country_widget = widgets.Dropdown(
    description='Country:',
    options=[country.name for country in pycountry.countries],
    value='Sao Tome and Principe',  # Pre-selected value
    layout=item_layout2,
    style={'description_width': 'initial'}
)

display(country_widget)

In [ ]:
if fetch_data:
  # If getting data from scratch
  un_boundaries = gpd.read_file("https://zstagigaprodeuw1.blob.core.windows.net/gigainframapkit-public-container/country_boundary_data/boundaries.geojson")
  country = un_boundaries[un_boundaries.romnam == country_widget.value]
else:
  # If using pre-loaded data
  country = gpd.read_file("https://zstagigaprodeuw1.blob.core.windows.net/gigainframapkit-public-container/stp/raw/stp.geojson")

In [ ]:
country.plot(color="green")

In [ ]:
boundary = country.total_bounds
utm = country.estimate_utm_crs()
latitude = country.centroid.y.squeeze()

In [ ]:
# @title Function
def get_iso3_country_code(country_name):
    try:
        country = pycountry.countries.get(name=country_name)
        return country.alpha_3
    except AttributeError:
        return None

In [ ]:
country_code = pycountry.countries.get(name = country_widget.value).alpha_3
print(f"The ISO-3 country code for {country_widget.value} is {country_code}.")

## Get point of interest (POI) data

<img src="https://wiki.openstreetmap.org/w/images/c/c8/Public-images-osm_logo.png" alt="OpenStreetMap logo" width="20%">

[OpenStreetMap](https://www.openstreetmap.org/) (OSM) is an open-source, community-driven geospatial data project that provides free and editable geographic information at a global scale. Established in 2004, OSM functions as a vast, collaborative mapping database where users contribute data on roads, buildings, land use, natural features, and more, creating a detailed digital representation of the world's infrastructure. Its open data licensing and API access make it highly valuable for applications in GIS analysis, urban planning, transportation modeling, and disaster response.

We can directly **query** OpenStreetMap online to look for schools using an online tool: https://overpass-turbo.eu/s/1XsE.

![osm-query](https://i.ibb.co/drqmqrP/Screenshot-2024-11-12-at-10-36-06.png)

Othwerise, we can also query the OpenStreetMap data using Python code. The code below:
1. Either fetches school data from OpenStreetMap using the `osmnx` API and the `amenity=school` tag (if `fetch_data`=`True`) or loads pre-downloaded data
2. Filters the data to keep only relevant columns (ID, amenity type, city, education level, etc.)
3. Processes the geographical coordinates for each school

This data will help us understand the distribution of educational facilities across the country and identify areas that might be underserved.

In [ ]:
if fetch_data:
    # If getting data from scratch
    place = country_widget.value
    tags = {"amenity": "school"}
    schools_gdf = ox.features_from_place(place, tags)
    schools_gdf = schools_gdf.reset_index(inplace=False)
else:
    # If using pre-loaded data
    schools_gdf = gpd.read_file("https://zstagigaprodeuw1.blob.core.windows.net/gigainframapkit-public-container/stp/raw/schools.geojson")
    schools_gdf = schools_gdf.reset_index(inplace=False)

In [ ]:
schools_gdf = schools_gdf.reset_index()[["id","amenity","operator", "geometry"]]

In [ ]:
schools_gdf.head()

In [ ]:
schools_gdf.plot()

## Get Ookla speed test data

<img src="https://i.ibb.co/wcB1JHC/Screenshot-2024-10-25-at-21-24-24.png" alt="Ookla Open Data" width="75%">

[Ookla](https://www.ookla.com/ookla-for-good/open-data), known for its Speedtest platform, offers open data on internet speeds, latency, and network quality worldwide. Their datasets, like the Speedtest Global Index, help researchers analyze internet performance and infer knowledge about infrastructure gaps.

Here, we process their internet speed test data. For both mobile and fixed broadband:
1. We load data from Ookla's public dataset hosted as parquet files on Amazon Web Services (or pre-downloaded files)
2. Filter the data to our geographical bounds obtained earlier
3. Fetch key metrics like average download speed (`avg_d_kbps`) and latency (`avg_lat_ms`)
4. For mobile data, we can also create coverage polygons by buffering around test points. The Ookla data is available as tiles which are approximately 610.8 meters by 610.8 meters at the equator.
Fore more information about processing open data from Ookla, visit their [GitHub](https://github.com/teamookla/ookla-open-data) page.

In [ ]:
# @title Function
def get_perf_tiles_parquet_url(service: str, year: int, quarter: int) -> str:
    quarter_start = f"{year}-{(((quarter - 1) * 3) + 1):02}-01"
    url = f"s3://ookla-open-data/parquet/performance/type={service}/year={year}/quarter={quarter}/{quarter_start}_performance_{service}_tiles.parquet"
    return url

### Mobile

In [ ]:
if fetch_data:
  # If getting data from scratch
  mobile_perf_tiles_url = get_perf_tiles_parquet_url("mobile", 2024, 2)
  bbox_filters = [('tile_y', '<=', boundary[3]), ('tile_y', '>=', boundary[1]),
                ('tile_x', '<=', boundary[2]), ('tile_x', '>=', boundary[0])]
  mobile_tiles_df = pd.read_parquet(mobile_perf_tiles_url,
                           filters=bbox_filters,
                           columns=['tile_x', 'tile_y', 'tests', 'avg_d_kbps', 'avg_lat_ms'],
                           storage_options={"s3": {"anon": True}}
                           )
else:
  # If using pre-loaded data
  mobile_tiles_df = pd.read_csv("https://zstagigaprodeuw1.blob.core.windows.net/gigainframapkit-public-container/stp/raw/stp-ookla-mobile-tiles.csv",index_col=0)

In [ ]:
mobile_tiles_gdf = gpd.GeoDataFrame(mobile_tiles_df, geometry=gpd.points_from_xy(mobile_tiles_df.tile_x, mobile_tiles_df.tile_y), crs="EPSG:4326").drop(columns=["tile_x", "tile_y"])

In [ ]:
mobile_tiles_gdf.head()

In [ ]:
mobile_tiles_gdf.plot()

#### Generate mobile coverage area

We can infer mobile coverage areas from Ookla's internet speed test data. We assume that areas where there have been succesful mobile speed tests are areas that have cellular coverage, and vice versa. We do not have information, however, on which cellular technology it refers to (3G, 4G, 5G).

In [ ]:
tile_size_at_latitude=610.8*np.cos(math.radians(latitude))
buffers = mobile_tiles_gdf.to_crs(utm).buffer(tile_size_at_latitude).to_crs("EPSG:4326")
single_polygon = unary_union(buffers)
mobile_coverage_gdf = gpd.GeoDataFrame(geometry=[single_polygon], crs="EPSG:4326")

In [ ]:
mobile_coverage_gdf.head()

In [ ]:
mobile_coverage_gdf.plot()

### Fixed

In [ ]:
if fetch_data:
  # If getting data from scratch
  fixed_perf_tiles_url = get_perf_tiles_parquet_url("fixed", 2024, 2)
  fixed_tiles_df = pd.read_parquet(fixed_perf_tiles_url,
                           filters=bbox_filters,
                           columns=['tile_x', 'tile_y', 'tests', 'avg_d_kbps', 'avg_lat_ms'],
                           storage_options={"s3": {"anon": True}}
                           )
else:
  # If using pre-loaded data
  fixed_tiles_df = pd.read_csv("https://zstagigaprodeuw1.blob.core.windows.net/gigainframapkit-public-container/stp/raw/stp-ookla-fixed-tiles.csv")

In [ ]:
fiber_nodes_gdf = gpd.GeoDataFrame(fixed_tiles_df, geometry=gpd.points_from_xy(fixed_tiles_df.tile_x,fixed_tiles_df.tile_y), crs="EPSG:4326")

In [ ]:
fiber_nodes_gdf.head()

In [ ]:
fiber_nodes_gdf.plot()

## Get cell site data

<img src="https://wiki.opencellid.org/images/d/de/OpenCellID_banner_main_page2.png" alt="Ookla Open Data" width="50%">

[OpenCellID](https://wiki.opencellid.org/wiki/What_is_OpenCellID) is the world's largest collaborative community project that collects GPS positions of cell towers, used free of charge, for a multitude of commercial and private purposes. Notably, they publish data on cell site coordinates. In order to download their data, register at their page and obtain a free API acces token. Using this token, you will be able to download [datasets](https://opencellid.org/downloads.php) for each country.

The code below:
1. Loads a CSV file containing cell site coordinates obtained from OpenCellID.
2. Converts it to a GeoDataFrame for spatial analysis

In [ ]:
# @title Function
def unzip_gz(gz_file_path, output_path=None):
    """
    Unzips a .gz file and returns the path to the unzipped file.

    Args:
        gz_file_path (str): Path to the .gz file
        output_path (str): Path for the output file.

    Returns:
        str: Path to the unzipped file
    """
    # Create output directory if it doesn't exist
    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Unzip the file
    with gzip.open(gz_file_path, 'rb') as f_in:
        with open(output_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    return output_path

In [ ]:
# Change this to True, if you are getting raw data from OpenCellID
use_open_cell_id = False

In [ ]:
if use_open_cell_id:
  # If getting raw data from OpenCellID, download the .csv.fz file from https://opencellid.org/downloads.php
  # You will need to register and get an API key, then search by country and download the file(s
  # Then upload them to your Google Drive, and copy the location of the file and assign it to location_of_gz_file
  location_of_gz_file = "/content/626.csv.gz"
  unzipped_cell_sites = unzip_gz(location_of_gz_file,"/content/cell_sites.csv")
  cell_sites = pd.read_csv(unzipped_cell_sites, header=None,
                         names = ['radio', 'mcc', 'net', 'area', 'cell', 'unit', 'lon', 'lat',
                                  'range', 'samples', 'changeable', 'created', 'updated', 'averageSignal'])
  cell_sites_gdf = gpd.GeoDataFrame(cell_sites, geometry=gpd.points_from_xy(cell_sites.lon, cell_sites.lat), crs="EPSG:4326").drop(columns=["lon", "lat"])
else:
  # If not using the Open Cell ID data for Sao Tome and Principe, use the data from Ookla as an approximation of cell site locations (inappropriate!!!)
  cell_sites_gdf = mobile_tiles_gdf

In [ ]:
cell_sites_gdf.head()

In [ ]:
cell_sites_gdf.plot()

# Standardize data

Finally, we standardize all our data into a consistent format, following the [ITU data dictionaries](https://bbmaps.itu.int/geonetwork/srv/eng/catalog.search;jsessionid=4BB00A9A95D58DCCAAD3967DC2DEA0E0#/metadata/d4fce2b9-ed20-4a3e-9312-4f04e1a384ad) for infrastructure data.
1. Create unified schemas for each infrastructure type (POIs, cell sites, nodes, coverage)
2. Generate unique IDs for each feature, using UUIDs (Universally Unique Identifier). These are long serial numbers that are almost guaranteed to be unique every time we generate them.
3. Transform data to match standard schemas
4. Save standardized data to CSV/GeoJSON files

This standardization makes it easier to:
- Share data with other analysts
- Perform consistent analysis across different regions and projects

In [ ]:
# @title Function
def extract_lat_lon(gdf, id_column='id'):
   """
   Create a new geodataframe with latitude, longitude and UUID columns
   """
   data = pd.DataFrame({
       id_column: [str(uuid.uuid4()) for _ in range(len(gdf))],
       'dataset_id': str(uuid.uuid4()),
       'lat': gdf.geometry.y,
       'lon': gdf.geometry.x,
   })
   gdf = gpd.GeoDataFrame(data, geometry=gdf.geometry, crs=gdf.crs)
   return gdf

## Point of interest (POI) data

In [ ]:
# @title POIs
poi_metadata = pd.DataFrame({
   'column_name': ['poi_id', 'dataset_id', 'lat', 'lon', 'poi_type', 'is_public', 'poi_subtype', 'country_code', 'is_connected', 'connectivity_type'],
   'column_type': ['UUID', 'UUID', 'float', 'float', 'string', 'boolean', 'string', 'string', 'boolean', 'string'],
   'levels': [''] * 10,
   'example': ['123e4567-e89b-12d3-a456-426614174000', '987fcdeb-51a2-12d3-a456-426614174000', '36.7538', '3.0588', 'school', 'True', 'primary school', 'DZA', 'True', '4G'],
   'mandatory': ['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'No'],
   'definition': [
       'Unique identifier for the POI',
       'Unique identifier for the dataset',
       'Latitude coordinate',
       'Longitude coordinate',
       'Type of point of interest',
       'Whether the POI is public or private',
       'Specific subtype of the POI',
       'ISO 3166-1 alpha-3 country code',
       'Whether the POI has connectivity',
       'Type of internet connectivity'
   ]
})
styled_df = poi_metadata.style.set_properties(**{
   'text-align': 'left',
   'border': '1px solid black',
   'padding': '8px'
}).set_table_styles([
   {'selector': 'thead', 'props': [('background-color', '#f2f2f2'), ('font-weight', 'bold'), ('border-bottom', '2px solid black')]},
   {'selector': 'tbody tr:nth-of-type(odd)', 'props': [('background-color', '#f9f9f9')]}
])
display(styled_df)

In [ ]:
# Convert geometries to centroids
schools_gdf.geometry = schools_gdf.geometry.centroid

# Create blank dataframe with id, latitute and longitude columns
formatted_schools = extract_lat_lon(schools_gdf, id_column='poi_id')

# Fill in other columns
formatted_schools["country_code"] = country_code
formatted_schools["poi_type"] = "school"
formatted_schools["is_connected"] = False

In [ ]:
formatted_schools.head()

In [ ]:
# Export file to use in later analysis
formatted_schools.to_file(f"/content/formatted_schools.geojson", driver="GeoJSON")

## Cell site data

In [ ]:
# @title Cell sites
cell_metadata = pd.DataFrame({
   'column_name': ['ict_id', 'dataset_id', 'latitude', 'longitude', 'operator_name', 'radio_type', 'antenna_height_m', 'backhaul_type', 'backhaul_throughput_mbps'],
   'column_type': ['UUID', 'UUID', 'float', 'float', 'string', 'string', 'float', 'string', 'float'],
   'levels': [
       '',  # ict_id
       '',  # dataset_id
       '',  # latitude
       '',  # longitude
       '',  # operator_name
       'LTE, UMTS, GSM, CDMA',  # radio_type
       '',  # antenna_height_m
       'fiber, microwave, satellite',  # backhaul_type
       ''   # backhaul_throughput_mbps
   ],
   'example': ['123e4567-e89b-12d3-a456-426614174000', '987fcdeb-51a2-12d3-a456-426614174000', '38.988755', '1.401938', 'TelOperator', 'LTE', '25', 'fiber', '1000'],
   'mandatory': ['Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No'],
   'definition': [
       'Cell tower identifier',
       'Unique identifier for the dataset',
       'Cell tower geographical latitude',
       'Cell tower geographical longitude',
       'Mobile network operator name',
       'Type of radio transmission technology',
       'Antenna height on the tower or building',
       'Type of backhaul connectivity of the cell tower',
       'Equipped throughput of the backhaul'
   ]
})
styled_df = cell_metadata.style.set_properties(**{
   'text-align': 'left',
   'border': '1px solid black',
   'padding': '8px'
}).set_table_styles([
   {'selector': 'thead', 'props': [('background-color', '#f2f2f2'), ('font-weight', 'bold'), ('border-bottom', '2px solid black')]},
   {'selector': 'tbody tr:nth-of-type(odd)', 'props': [('background-color', '#f9f9f9')]}
])
display(styled_df)

In [ ]:
cell_sites_gdf

In [ ]:
if use_open_cell_id:
  cell_sites_gdf["radio"].value_counts()

We assume that each antenna height is 25 meters.

In [ ]:
# Create blank dataframe with id, latitute and longitude columns
formatted_cell_sites = extract_lat_lon(cell_sites_gdf, id_column='ict_id')

# Fill in other columns
formatted_cell_sites["radio_type"] = cell_sites_gdf["radio"] if use_open_cell_id else "LTE"
formatted_cell_sites["antenna_height_m"] = 25
formatted_cell_sites["backhaul_type"] = pd.NA
formatted_cell_sites["backhaul_throughput_mbps"] = pd.NA
formatted_cell_sites["operator_name"] = pd.NA

In [ ]:
formatted_cell_sites.head()

In [ ]:
# Export file to use in later analysis
formatted_cell_sites.to_file(f"/content/formatted_cell_sites.geojson", driver="GeoJSON")

## Transmission node data

In [ ]:
# @title Nodes
node_metadata = pd.DataFrame({
   'column_name': ['ict_id', 'dataset_id', 'latitude', 'longitude', 'operator_name', 'infrastructure_type', 'node_status', 'equipped_capacity_mbps', 'potential_capacity_mbps'],
   'column_type': ['UUID', 'UUID', 'float', 'float', 'string', 'string', 'string', 'float', 'float'],
   'levels': [
       '',  # node_id
       '',  # dataset_id
       '',  # latitude
       '',  # longitude
       '',  # operator_name
       'fiber, microwave, other',  # infrastructure_type
       'operational, planned, under construction',  # node_status
       '',  # equipped_capacity_mbps
       ''   # potential_capacity_mbps
   ],
   'example': ['123e4567-e89b-12d3-a456-426614174000', '987fcdeb-51a2-12d3-a456-426614174000', '38.988755', '1.401938', 'TelOperator', 'fiber', 'operational', '1000', '2000'],
   'mandatory': ['Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No'],
   'definition': [
       'Node identifier',
       'Unique identifier for the dataset',
       'Geographical latitude',
       'Geographical longitude',
       'Name of the mobile operator',
       'Type of Infrastructure',
       'Status of the node',
       'Equipped bandwidth ready for use to connect subscribers',
       'Total theoretical bandwidth available for subscriber connections'
   ]
})

styled_df = node_metadata.style.set_properties(**{
   'text-align': 'left',
   'border': '1px solid black',
   'padding': '8px'
}).set_table_styles([
   {'selector': 'thead', 'props': [('background-color', '#f2f2f2'), ('font-weight', 'bold'), ('border-bottom', '2px solid black')]},
   {'selector': 'tbody tr:nth-of-type(odd)', 'props': [('background-color', '#f9f9f9')]}
])
display(styled_df)

All our points are fiber nodes, and we assume that they are all operational.

In [ ]:
# Create blank dataframe with id, latitute and longitude columns
formatted_nodes = extract_lat_lon(fiber_nodes_gdf, id_column='ict_id')

# Fill in other columns
formatted_nodes["operator_name"] = pd.NA
formatted_nodes["infrastructure_type"] = "fiber"
formatted_nodes["node_status"] = "operational"
formatted_nodes["equipped_capacity_mbps"] = pd.NA
formatted_nodes["potential_capacity_mbps"] = pd.NA

In [ ]:
formatted_nodes.head()

In [ ]:
# Export file to use in later analysis
formatted_nodes.to_file(f"/content/formatted_nodes.geojson", driver="GeoJSON")

## Mobile coverage

In [ ]:
# @title Coverage
coverage_metadata = pd.DataFrame({
   'column_name': ['coverage_id', 'dataset_id', 'signal_strength_dbm', 'operator_name', 'geometry', 'coverage'],
   'column_type': ['UUID', 'UUID', 'float', 'string', 'geometry', 'integer'],
   'levels': [
       '',  # coverage_id
       '',  # dataset_id
       '',  # signal_strength
       '',  # operator_name
       'polygon',  # geometry
       '1'
   ],
   'example': [
       '123e4567-e89b-12d3-a456-426614174000',
       '987fcdeb-51a2-12d3-a456-426614174000',
       '-93',
       'TelOperator',
       'POLYGON((...))',
       '1'
   ],
   'mandatory': ['Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes'],
   'definition': [
       'Unique identifier for the coverage area',
       'Unique identifier for the dataset',
       'Mobile signal strength in dBm for coverage',
       'Name of the mobile operator',
       'Polygon geometry of coverage area',
       'Binary value indicating coverage'
   ]
})

styled_df = coverage_metadata.style.set_properties(**{
   'text-align': 'left',
   'border': '1px solid black',
   'padding': '8px'
}).set_table_styles([
   {'selector': 'thead', 'props': [('background-color', '#f2f2f2'), ('font-weight', 'bold'), ('border-bottom', '2px solid black')]},
   {'selector': 'tbody tr:nth-of-type(odd)', 'props': [('background-color', '#f9f9f9')]}
])
display(styled_df)

In [ ]:
# Create blank dataframe with id, latitute and longitude columns
formatted_coverage = mobile_coverage_gdf

# Fill in other columns
formatted_coverage["coverage"] = 1
formatted_coverage["signal_strength_dbm"] = pd.NA
formatted_coverage["operator_name"] = pd.NA
formatted_coverage["coverage_id"] = [str(uuid.uuid4()) for _ in range(len(formatted_coverage))]
formatted_coverage["dataset_id"] = str(uuid.uuid4())

In [ ]:
formatted_coverage.head()

In [ ]:
# Export file to use in later analysis
formatted_coverage.to_file(f"/content/formatted_coverage.geojson", driver="GeoJSON")